# Data from CSV

In [0]:
dbutils.fs.ls("dbfs:/databricks-datasets/bikeSharing/data-001/day.csv")

In [0]:
# dbutils.fs.ls("/FileStore/tables/Electric_Vehicle_Population_Data.csv")

# dbutils.fs.rm("/FileStore/tables/Electric_Vehicle_Population_Data.csv")


In [0]:
df= spark.read.format('csv').option('header', 'true').\
option('InferSchema', 'true').load('dbfs:/databricks-datasets/bikeSharing/data-001/day.csv')

In [0]:
# df.count()
df.display()

In [0]:
from pyspark.sql.functions import *

df1 = df.groupBy(
    year(col("dteday")).alias("year"),
    month(col("dteday")).alias("month")
).agg(
    count("*").alias("record_count")
).orderBy("year", "month")

In [0]:
# df1 = df1.withColumn(
#     "Month_Name",
#     date_format(
#         to_date(lpad(col("month").cast("string"), 2, "0"), "MM"),
#         "MMMM"
#     )
# )

# df1.display()
df1.select( "year","Month_Name", "record_count").write.format("delta").mode("overwrite").saveAsTable("databricks_demo.BikeSharingcount")


## Mount Azure

<!-- #Select * from databricks_demo.bikesharingcount

spark.sql("Select * from databricks_demo.bikesharingcount").display()

spark.sql("Select year from databricks_demo.bikesharingcount").display()

# spark.sql("drop table databricks_demo.bikesharingcount").display()

# df1.display() -->


In [0]:
%sql
create volume if not exists Volumestest

In [0]:
# File location and type
file_location = "/FileStore/tables/Electric_Vehicle_Population_Data.csv"
file_type = "csv"

# CSV options
infer_schema = "true"
first_row_is_header = "true"
delimiter = ","

# The applied options are for CSV files. For other file types, these will be ignored.
df_ele = spark.read.format(file_type) \
  .option("inferSchema", infer_schema) \
  .option("header", first_row_is_header) \
  .option("sep", delimiter) \
  .load(file_location)

# display(df_ele)


In [0]:
new_col = [i.replace(" ", "_").replace("(", "").replace(")", "").replace("-", "_") for i in df_ele.columns]

df_ele = df_ele.toDF(*new_col)

df_ele.write.format("delta").mode("overwrite").saveAsTable("databricks_demo.ElectricVehicle_Info")

In [0]:
staging = spark.sql("""
SELECT * FROM databricks_demo.ElectricVehicle_Info
WHERE County = 'Snohomish' AND City = 'Everett' AND Postal_Code = 98204
""")
staging.write.format("delta").mode("overwrite").saveAsTable("databricks_demo.ElectricVehicle_Info_P98204")

In [0]:
transfrm_tbl = spark.read.table("databricks_demo.ElectricVehicle_Info")
transfrm_tbl.display()



In [0]:
from pyspark.sql.functions import *

In [0]:
transfrm_tbl.groupBy("DOL_Vehicle_ID").agg(count('*').alias("cnt")).filter("cnt > 1").select("DOL_Vehicle_ID","cnt").display()

In [0]:
staging = staging.where(
    (col("County").like("%Snohomish")) &
    (col("City").like("%Everett")) &
    (col("Postal_Code") == 98204)
)

In [0]:
from pyspark.sql.functions import *

var = df_ele.where(
    (col("County").like("%Snohomish")) &
    (col("City").like("%Everett")) &
    (col("Postal Code") == 98204)
)
var.display()
# var.write.format("delta").mode("overwrite").saveAsTable("databricks_demo.ElectricVehicle_Info_P98204")

In [0]:
var.columns

In [0]:
#  for i in var.columns:
#     print(i.replace(" ", "_").replace("(", "").replace(")", "").replace("-", "_"))
new_col = [i.replace(" ", "_").replace("(", "").replace(")", "").replace("-", "_") for i in var.columns]

var = var.toDF(*new_col)
#var.display()
var.write.format("delta").mode("overwrite").saveAsTable("databricks_demo.ElectricVehicle_Info_P98204")


In [0]:
%sql
select * from databricks_demo.electricvehicle_info_p98204;

In [0]:
from delta.tables import DeltaTable

if True:
    # Use DeltaTable merge
    cols = transfrm_tbl.columns
 
    update_expr = {col: f"s.{col}" for col in cols}
    insert_expr = {col: f"s.{col}" for col in cols}
    print(update_expr)
    print(insert_expr)

    target = DeltaTable.forName(spark, "databricks_demo.ElectricVehicle_Info_P98204")

    target.alias("t").merge(
        transfrm_tbl.alias("s"),
        "t.DOL_Vehicle_ID = s.DOL_Vehicle_ID and t.VIN_1_10 = s.VIN_1_10 AND t.Model_Year = s.Model_Year and t.County = s.County AND t.City = s.City AND t.Postal_Code = s.Postal_Code"
    )\
    .whenMatchedUpdate(set=update_expr) \
    .whenNotMatchedInsert(values=insert_expr) \
    .execute()


 

In [0]:
# spark.sql("truncate table databricks_demo.ElectricVehicle_Info_P98204");
spark.sql("truncate table databricks_demo.ElectricVehicle_Info");

In [0]:
%sql
-- select count(1) from databricks_demo.electricvehicle_info_p98204
-- union all
-- select count(1) from databricks_demo.electricvehicle_info;

Select * from databricks_demo.electricvehicle_info_p98204 where State like '%tk'
-- Select * from databricks_demo.electricvehicle_info

In [0]:
df_CSV = spark.sql(
    "select * from databricks_demo.electricvehicle_info_p98204 "
    "union all "
    "(select VIN_1_10 || '1', County, City, State || 'tk', Postal_Code, "
    "Model_Year, Make, Model, Electric_Vehicle_Type, "
    "Clean_Alternative_Fuel_Vehicle_CAFV_Eligibility, Electric_Range, Base_MSRP, "
    "Legislative_District, DOL_Vehicle_ID, Vehicle_Location, Electric_Utility, "
    "2020_Census_Tract from databricks_demo.electricvehicle_info_p98204 Limit 3)"
)

In [0]:
%sql
select * from databricks_demo.electricvehicle_info where State like '%tk'

In [0]:
# df_CSV.write.format("csv").mode("overwrite").save("/FileStore/tables/electricvehicle_info_p98204_Test.csv")
df_CSV.display()
# dbutils.fs.ls("/FileStore/tables/")